### Dash er ett dashboardingverktøy fra folka hos Plotly
Den bruker React-komponenter for å lage interaktive dashboards i browseren.\
Dette eksempelet er basert på Simen Svenkeruds eksperiment i freg,\
CSS fra Mimir-prosjektet og noe input fra denne youtubevideoen: https://www.youtube.com/watch?v=hSPmj7mK6ng \
All javascript og CSS plassert på samme nivå som notebooken, i en mappe som heter "assets, vil lastes inn.\
Så dette eksempelet laster faktisk assets/style.css, som er hentet herfra: https://github.com/statisticsnorway/ssb-component-library/blob/master/lib/bundle.css

### (English) Dash is a dashboarding tool from the guys at Plotly
It uses React components to create interactive dashboards in the browser.\
This example is based on Simen Svenkeruds experiments in freg,\
CSS from the Mimir-project and some input from this youtube-video: https://www.youtube.com/watch?v=hSPmj7mK6ng \
All javascript and CSS placed next to the executing script in an "assets"-folder, will be loaded.\
So this actually loads the assets/style.css, which was taken from here: https://github.com/statisticsnorway/ssb-component-library/blob/master/lib/bundle.css

In [ ]:
import pandas as pd
import dapla as dp

In [ ]:
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Merk deg denne, da vi caller den inni funksjonen lengre nede på px.scatter
import plotly.express as px

In [ ]:
from jupyter_dash import JupyterDash
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
# This might struggle to rerun, if you do not restart your server in the Server Hub Control
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# Kanskje unødvendig, men greit å ha om man sliter med å ta ned serveren etter at den er startet, 
# kjøres fra siste celle for å terminere lokal-serveren som Dash kjører, om ønskelig.
@classmethod
def _terminate_server_for_port(cls, host, port):
    shutdown_url = "http://{host}:{port}/_shutdown_{token}".format(
        host=host, port=port, token=JupyterDash._token
    )
    try:
        response = requests.get(shutdown_url)
    except Exception as e:
        pass

In [ ]:
# Les eksempeldatasett fra dapla
diam = dp.read_pandas("/felles/kurs/helt_python/diamonds")

In [ ]:
# Generer liste med "options" til dropdownen lengre nede, må være formatert i dicts
drop_options = [{'label': 'Alle', 'value' : 'Alle'}] # Første alternativ, som inneholder alle farger

# Legg inn resten av farger, basert på data i dataframen
diam_colors = pd.unique(diam['color'])
for color in diam_colors:
    tmp_dict = {"label" : color, "value": color} # Dropdown-options krever at de er formatert slik, som dicts
    drop_options.append(tmp_dict) # Legg til hver farge-options-dict i listen over options
    
# Uncomment om du vil se resultatet:
#drop_options

In [ ]:
# Opprett app-objektet
app = JupyterDash(__name__)

# Her er alle html-elementene som inngår i Dash-dashboardet.
# Finn passende "className" fra style.css som gjerne kan tas herfra 
# https://github.com/statisticsnorway/ssb-component-library/blob/master/lib/bundle.css
app.layout = html.Div([
    # Overskrift, className er hentet fra CSSen fra Mimir-prosjektet
    html.H1('Diamanter Dash-eksempel', className = "ssb-title"),
    
    # Dropdown for valg av "farge"
    dcc.Dropdown('slct_color', # Denne tittelen brukes som input i funksjonen lengre ned, så det er viktig det stemmer med hverandre
                 options = drop_options, # Denne bruker da settet med options vi lagde i cellen lengre opp
                 multi = False,
                 value = "Alle", # Samme som første option, se lengre opp
                 style = {'width' : '40%'}, # Kunne også vært definert i css-filen
                 className = "ssb-dropdown" # className er hentet fra CSSen fra Mimir-prosjektet
                ),
    
    # Denne skal holde på tekstbiten som settes i "container", lengre ned
    html.Div(id = 'output_container', children =[]),
    # Dette er kun en kort "break", dvs. html-linjeskift
    html.Br(),
    # Her setter vi inn innhold for grafen, som kommer som output av funksjonen lengre nede.
    dcc.Graph(id="diam_scatter", figure={})    
    
])

In [ ]:
# Dette er en decorator, som går "utenpå" funksjonen under, 
# som både lager grafen, og tar imot oppdateringer ift. input.
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='diam_scatter', component_property='figure')],
    [Input(component_id='slct_color', component_property='value')]
)
# Definer funksjonen som tar oppdateringer på inputen. 
# Omkranses av decoratoren over.
def update_graph(option_slctd):
    
    # Enkelt varsel om hvilket alternativ du har valgt
    container = f"Du har valgt fargen: {option_slctd}"
    
    # Ikke kødd med orginaldataframen inni en funksjon
    dff = diam.copy()
    
    # Filtrer om noe annet enn "All" er valgt 
    if option_slctd != "Alle":
        dff = dff[dff['color'] == option_slctd]
        
    # Ett scatterplot fra plotly.express
    fig = px.scatter(
        data_frame = dff,
        x = 'price',
        y = 'carat',
        color = 'color',
        # SSB-fargeskala
        color_discrete_sequence=["#1a9d49", "#3396d2", "#9582bb", "#f26539", "#d2bc2a", 
                                 "#274247", "#90cc93", "#0757450", "#143f90", "#472f91",
                                 "#93180a", "#9a7b1c", "#6f9090"]
    )
    
    # Vi må returnere ett objekt per output listet i decoratoren i toppen,
    # så om du endrer antall outputs, må du også endre her nede.
    return container, fig

In [ ]:
# Start serveren på spesifisert port, returnerer en linktekst
app.run_server(mode='external', port=8060)

In [ ]:
# This isnt really enough to rerun the scripts, as there will be issues with regestering outputs with the callback decorator etc.
#app._terminate_server_for_port("localhost", 8060)